In [1]:
import numpy as np
import pandas as pd
from cleanlab import Datalab
from transformers import pipeline

In [2]:
import fasttext
from tqdm.auto import tqdm

encoder = fasttext.load_model("./encoders/cc.ar.300.bin")

In [3]:
dataset = pd.read_excel("./data/to_label.xlsx", sheet_name="Sheet1")
dataset["label"] = dataset["label"].astype(int)
dataset.head()

,tweetid,tweet_text,clean_tweet,tech,label
0,295735762505322497,النعيم أمي، وعين أمي، وقبلة أمي، فأكرمها يالله...,النعيم أمي، وعين أمي، وقبلة أمي، فأكرمها يالله...,NaN,0
1,1080873494227886086,RT @rood516: عرض الاجازه😍👌🏻1\nســــاعه\nقـــلم...,RT : عرض الاجازه1 ســــاعه قـــلم كــبك نظـــا...,NaN,0
2,1027481525506453504,RT @6FSH1_: مهما ضاق صدرك تذكر أن في ناس توهم ...,RT : مهما ضاق صدرك تذكر أن في ناس توهم بيسجلون...,NaN,0
3,417202038037299200,RT @f450450: ﴿ياحي ياقيوم برحمتك أستغيث أصلح ل...,RT : ﴿ياحي ياقيوم برحمتك أستغيث أصلح لي شأني ك...,NaN,0
4,834074569673736193,RT @7744Qa: بعض البشر ، إذا وجد البديل نكر الج...,RT : بعض البشر ، إذا وجد البديل نكر الجميل. قر...,NaN,0


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweetid      2100 non-null   int64 
 1   tweet_text   2100 non-null   object
 2   clean_tweet  2100 non-null   object
 3   tech         202 non-null    object
 4   label        2100 non-null   int32 
dtypes: int32(1), int64(1), object(3)
memory usage: 74.0+ KB


In [5]:
tweets = dataset.clean_tweet.to_list()
labels = dataset.label.values

In [6]:
features = []

for tweet in tqdm(tweets):
    features.append(encoder.get_sentence_vector(tweet))

  0%|          | 0/2100 [00:00<?, ?it/s]

In [7]:
data = {"X": np.array(features), "Y": labels}

In [8]:
model = pipeline(
    task="text-classification", model="./models/arabertv2-propaganda/", device=0
)

In [9]:
outs = model(tweets)

In [10]:
label_names = ["transparent", "propaganda"]

In [11]:
pred_probs = []
for pred in outs:
    i = label_names.index(pred["label"])
    logits = [None] * len(label_names)
    logits[i] = pred["score"]
    logits[len(label_names) - 1 - i] = 1 - pred["score"]
    pred_probs.append(logits)

In [12]:
pred_probs = np.array(pred_probs)

In [13]:
lab = Datalab(data, label_name="Y")
lab.find_issues(pred_probs=pred_probs, features=data["X"])

Finding label issues ...
Finding outlier issues ...
Fitting OOD estimator based on provided features ...
Finding near_duplicate issues ...
Audit complete. 110 issues found in the dataset.


In [14]:
lab.report()

Here is a summary of the different kinds of issues found in the data:

    issue_type  num_issues
near_duplicate          46
       outlier          41
         label          23

Dataset Information: num_examples: 2100, num_classes: 2


------------------ near_duplicate issues -------------------

About this issue:
	A (near) duplicate issue refers to two or more examples in
    a dataset that are extremely similar to each other, relative
    to the rest of the dataset.  The examples flagged with this issue
    may be exactly duplicated, or lie atypically close together when
    represented as vectors (i.e. feature embeddings).
    

Number of examples with this issue: 46
Overall dataset quality in terms of this issue: 0.2115

Examples representing most severe instances of this issue:
      is_near_duplicate_issue  near_duplicate_score                                  near_duplicate_sets  distance_to_nearest_neighbor
1989                     True              0.003737                  

In [15]:
pd.set_option("display.max_colwidth", None, "display.max_rows", None)

In [16]:
issues = lab.get_issues()

In [17]:
issues.head()

,is_label_issue,label_score,is_outlier_issue,outlier_score,is_near_duplicate_issue,near_duplicate_score
0,False,0.885082,False,0.720818,False,0.272238
1,False,0.983625,False,0.808768,False,0.178953
2,False,0.982394,False,0.757022,False,0.259652
3,False,0.992450,False,0.785526,False,0.128088
4,False,0.987089,False,0.854087,False,0.120545


In [18]:
label_issues = dataset[issues.is_label_issue]
label_issues.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23 entries, 172 to 1970
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweetid      23 non-null     int64 
 1   tweet_text   23 non-null     object
 2   clean_tweet  23 non-null     object
 3   tech         3 non-null      object
 4   label        23 non-null     int32 
dtypes: int32(1), int64(1), object(3)
memory usage: 1012.0+ bytes


In [19]:
label_issues.head(100)

,tweetid,tweet_text,clean_tweet,tech,label
172,1180389807073177600,RT @aboody_s: #سعود_الصرامي\n\nمتحدثهم يقول !! https://t.co/bieGSPJ5yQ,RT : سعود الصرامي متحدثهم يقول !!,NaN,0
198,857169121334747136,RT @Aldowayesh: افعلهايامعالي رئيس الهيئة العامة للرياضة وافتح الملف المالي للرياضة ولاتأخذك في الله لومةلائم ولاوجاهة وجيه أومكانة سفيه…ن…,RT : افعلهايامعالي رئيس الهيئة العامة للرياضة وافتح الملف المالي للرياضة ولاتأخذك في الله لومةلائم ولاوجاهة وجيه أومكانة سفيه…ن…,NaN,0
264,1114176366616956928,RT @OKAZ_online: #عكاظ (الصفحة الأولى) #الجمعة:\n\n- هل يحسم #مجلس_الشورى #توظيف المرأة في «المحاكم».. أم يتعثر؟\n \n- ما هي الثغرات التي يستغل…,RT : عكاظ (الصفحة الأولى) الجمعة: - هل يحسم مجلس الشورى توظيف المرأة في «المحاكم».. أم يتعثر؟ - ما هي الثغرات التي يستغل…,NaN,0
301,1057511353072734213,RT @jwdya2681: #عطنا_صوره_لجو_مدينتك ما بعد المدينه مدينه ولا بعد المدينه جمال 😍💙💙 https://t.co/vWdMufXmiX,RT : عطنا صوره لجو مدينتك ما بعد المدينه مدينه ولا بعد المدينه جمال,exaggeration - loaded language,1
373,1119981960900878336,RT @Qat: وتستمر #الخطوط_الجوية_القطرية بحصد الجوائز.. https://t.co/3HU8cNcpoJ,RT : وتستمر الخطوط الجوية القطرية بحصد الجوائز..,name-calling - loaded language,1
424,997047670643527680,RT @dode_doo: #اعلان_زين استغفرالله العظيم وأتوب إليه,RT : اعلان زين استغفرالله العظيم وأتوب إليه,smears,1
586,768951335416922113,RT @twasulnews: المتهم بخطف رجال الأعمال «آل سند» في مصر يقتل نفسه أثناء التحقيق\n\nhttps://t.co/vYMd7txoNg\n\n#مصر #السعودية https://t.co/ps5p…,RT : المتهم بخطف رجال الأعمال «آل سند» في مصر يقتل نفسه أثناء التحقيق مصر السعودية,NaN,0
657,350754747860533248,#نايف_هزازي_الى_الشباب\n بالتوفيق لمن حرم #الاهلي من الدوري مع الشباب بعيد عن الاتحاد \nوان شاء الله يجيبلكم اسيا,نايف هزازي الى الشباب بالتوفيق لمن حرم الاهلي من الدوري مع الشباب بعيد عن الاتحاد وان شاء الله يجيبلكم اسيا,NaN,0
823,495816144184283137,#لؤي_ناظر_يحذف_الفضيحة\nنبي الدوري وولي العهد \nوهبوط الطحالب لركاء \nافعلهااا لنذكرك بالخير,لؤي ناظر يحذف الفضيحة نبي الدوري وولي العهد وهبوط الطحالب لركاء افعلهااا لنذكرك بالخير,NaN,0
902,1134525807173308416,اللهم لك الحمد حتى ترضى\n #القمممه_الاسلامية_بمكه,اللهم لك الحمد حتى ترضى القمممه الاسلامية بمكه,NaN,0


In [20]:
df = dataset[["tweetid", "tweet_text", "clean_tweet", "tech", "label"]].copy()
df = df.rename(columns={"clean_tweet": "text"})
df.head()

,tweetid,tweet_text,text,tech,label
0,295735762505322497,النعيم أمي، وعين أمي، وقبلة أمي، فأكرمها يالله بطول العمر بالصحه والعافية وازرع في قلبها سعادة لاتزول ♥,النعيم أمي، وعين أمي، وقبلة أمي، فأكرمها يالله بطول العمر بالصحه والعافية وازرع في قلبها سعادة لاتزول,NaN,0
1,1080873494227886086,RT @rood516: عرض الاجازه😍👌🏻1\nســــاعه\nقـــلم\nكــبك\nنظـــاره\nسبــحه 📿\nمـــيداليه\nخاتم\nمحفظه\nضمان الساعه لمدة سنه\nفقط ٢٢٠ريال\n🌟🌟\nواتس للطلب ا…,RT : عرض الاجازه1 ســــاعه قـــلم كــبك نظـــاره سبــحه مـــيداليه خاتم محفظه ضمان الساعه لمدة سنه فقط ٢٢٠ريال واتس للطلب ا…,NaN,0
2,1027481525506453504,RT @6FSH1_: مهما ضاق صدرك تذكر أن في ناس توهم بيسجلون بالجامعة.,RT : مهما ضاق صدرك تذكر أن في ناس توهم بيسجلون بالجامعة.,NaN,0
3,417202038037299200,RT @f450450: ﴿ياحي ياقيوم برحمتك أستغيث أصلح لي شأني كُلہ ولا تَكِلْني إلى نفسي طرفہ عين﴾,RT : ﴿ياحي ياقيوم برحمتك أستغيث أصلح لي شأني كلہ ولا تكلني إلى نفسي طرفہ عين﴾,NaN,0
4,834074569673736193,RT @7744Qa: بعض البشر ، إذا وجد البديل نكر الجميل.\n#قروب_شوشي_للدعم \n#قروب_همس_المشاعر \n#قروب_سججات_بدوي \n#قروب_السمو_للدعم,RT : بعض البشر ، إذا وجد البديل نكر الجميل. قروب شوشي للدعم قروب همس المشاعر قروب سججات بدوي قروب السمو للدعم,NaN,0


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweetid     2100 non-null   int64 
 1   tweet_text  2100 non-null   object
 2   text        2100 non-null   object
 3   tech        202 non-null    object
 4   label       2100 non-null   int32 
dtypes: int32(1), int64(1), object(3)
memory usage: 74.0+ KB


In [22]:
df.to_json("./data/to_label.json")